In [1]:
import argparse
import time
import socket
import torch
import random
import traceback
import numpy as np
import pandas as pd
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import json

In [2]:
def execute_gpt4(client, model_name, content):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": content}
        ],
        seed=53,
        max_tokens=128,
        temperature=0
    )
    rating = response.choices[0].message.content.strip()

    return rating

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
def process_json_file(client):
    # input_json_file = args.input_json_file
    testing_set = "postflop"
    input_json_file = "../data/poker-postflop/postflop_10k_test_set_prompt_and_label.json"
    model_name = "gpt-4o"

    print(f"Processing input JSON file: {input_json_file}")

    # Read JSON data
    with open(input_json_file, 'r') as f:
        data = json.load(f)

    # # For debugging purpose
    data = data[0:5000]

    ground_truths = []
    predictions = []
    for i in tqdm(range(len(data))):
        item = data[i]
        # Extract instruction (poker scenario) and expected output (optimal action)
        instruction = item.get('instruction', '')
        expected_output = item.get('output', '')
        ground_truths.append(expected_output)

        if instruction:  # Skip entries that don't have instructions
            # print(f"Processing poker scenario {len(predictions) + 1}")
            # print(instruction)

            try:
                # Generate model's response to the poker scenario
                model_response = execute_gpt4(client, model_name, instruction)


                # # Create result dictionary
                # result = {
                #     'instruction': instruction,
                #     'expected_output': expected_output,
                #     'model_response': model_response,
                #     'correct': model_response.strip() == expected_output.strip()
                # }

            except Exception as e:
                model_response = 'ERROR'
                # result = {
                #     'instruction': instruction,
                #     'expected_output': expected_output,
                #     'model_response': 'ERROR',
                #     'correct': False
                # }
                print(f"Failed to process poker scenario")
                print(f"Error: {e}")
                print(traceback.format_exc())

            predictions.append(model_response)

    # Create output dataframe
    results_df = pd.DataFrame({
        "Prediction": predictions,
        "Ground Truth": ground_truths
    })

    output_path = "../testing-results"
    # output_path = "./"
    # Save the DataFrames to CSV files
    results_df.to_csv(
        f"{output_path}/{model_name}-{testing_set}_predictions.csv", index=False)

In [ ]:
def main():
    client = OpenAI(api_key='')
    # process_json_file(client, args)
    process_json_file(client)

In [6]:

if __name__ == "__main__":
    # parser = argparse.ArgumentParser(
    #     description="Sentiment Transfer using LLM ChatGPT")
    # parser.add_argument('--model_name', type=str,
    #                     required=True, help='Pretrained LLM name',
    #                     default='gpt-3.5-turbo')
    # parser.add_argument('--input_json_file', required=True,
    #                     help="Path to the input json file",
    #                     default='../data/poker-preflop/preflop_1k_test_set_prompt_and_label.json')
    # args = parser.parse_args()

    start_time = time.time()
    # print("Command-line arguments:")
    # for arg, value in vars(args).items():
    #     print(f"{arg}: {value}")

    device = torch.device('mps' if torch.backends.mps.is_available() else (
        'cuda' if torch.cuda.is_available() else 'cpu'))
    print(f'Device {device}')

    # if torch.cuda.is_available():
    #     print(f'GPU machine name: {torch.cuda.get_device_name(0)}')
    #     print(f'SSH machine name: {socket.gethostname()}')

    set_seed(53)

    # main(args)
    main()

    end_time = time.time()
    time_taken_seconds = end_time - start_time
    time_taken_formatted = time.strftime(
        '%H:%M:%S', time.gmtime(time_taken_seconds))

    print(f"Time taken: {time_taken_formatted}")

    print('=' * 50 + '\n')

Device mps
Processing input JSON file: ../data/poker-postflop/postflop_10k_test_set_prompt_and_label.json


100%|██████████| 5000/5000 [58:10<00:00,  1.43it/s]  

Time taken: 00:58:10

